In [1]:
import os
import random
import numpy as np
import pandas as pd
import multiprocessing
import tensorflow as tf
import keras
from scipy import signal
from itertools import starmap
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from model_src.Resnet import ResNet34
from model_src.Unet import Unet
from model_src.DilatedConv import RespDNN
from model_src.BianResnet import ResNet
import matplotlib.pyplot as plt
print(f'Is GPU Avaliable: {tf.config.list_physical_devices("GPU")}')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
DATA_PATH = '../../DataLake/stMary'
kf = KFold(n_splits=5)

2023-06-07 18:48:44.082664: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-07 18:48:44.127656: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 18:48:44.766534: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
def generate_dataset(arg_pleths, arg_resps, fs=125, shift_factor=4):
    import copy
    dataset = []
    window_size = fs * 60 # 7500
    shift = int(window_size/shift_factor)
    samples_len = len(arg_pleths)

    cpy_resps = copy.deepcopy(arg_resps)
    cpy_pleths = copy.deepcopy(arg_pleths)

    for j in range(samples_len):
        rr = cpy_resps[j]; ppg = cpy_pleths[j]

        rr['offset'] = (rr['offset']-rr['offset'].min())/1000
        size_lim = int(fs * np.ceil(rr['offset'].max()))
        ppg = ppg[:size_lim]
        shift_n_times = int((len(ppg)-window_size)/shift)+1

        samp_rr = [len(rr.loc[ (rr['offset']>=0+(int(shift/fs)*i)) & ((rr['offset']<int(window_size/fs)+(int(shift/fs)*i))) ]) for i in range(shift_n_times)]
        samp_ppg = [ppg[0+(shift*i):window_size+(shift*i)] for i in range(shift_n_times)]

        for i in range(len(samp_ppg)):
            temp = []
            temp.append(samp_ppg[i])
            temp.append(samp_rr[i])
            dataset.append(temp)

    return dataset


def interpolation(x, input):
    x0 = int(np.floor(x))
    y0 = input[x0]
    x1 = int(np.ceil(x))
    y1 = input[x1]
    y = (y1-y0)*(x-x0) + y0
    return y


def signal_resample(input_signal, org_fs, new_fs, method='interpolation'):
    output_signal = []
    new_x = np.arange(0, len(input_signal), org_fs/new_fs)
    
    if method == 'interpolation': 
        interp = interpolation

    for x in new_x:
        y = interp(x, input_signal)
        output_signal.append(y)

    return np.asarray(output_signal)


def preprocessing(targets=None):
    print('Extract PLETH/RESP')
    pleths = [pd.read_csv(f'{DATA_PATH}/{sid}/pleth.csv', header=None, names=['sid', 'offset', 'pleth']).pleth.values for sid in targets.id.unique()]
    resps = [pd.read_csv(f'{DATA_PATH}/{sid}/respirationTimeline.csv', header=None, names=['sid', 'offset']) for sid in targets.id.unique()]

    # Before filtering: Check NaN
    for pleth in pleths:
        if any(np.isnan(pleth)):
            print('check')

    # Before filtering: Convert type as np.int16
    pleths = list(map(lambda pleth: pleth.astype(np.float32), pleths))


    print('Init Preprocessing: Filtering')
    taps = signal.firwin(numtaps=1000, cutoff=[0.5, 8.0], window='hamming', pass_zero=False, fs=125)
    w, h = signal.freqz(taps)
    pool = multiprocessing.Pool(processes=40)
    filtered_pleths = pool.starmap(signal.filtfilt, [(taps, 1.0, pleth) for pleth in pleths])
    pool.close()
    pool.join()


    print('Init Preprocessing: Windowing')
    dataset = generate_dataset(filtered_pleths, resps, shift_factor=60)


    print('Init Preprocessing: Resampling')
    pool = multiprocessing.Pool(processes=40)
    result = pool.starmap(signal_resample, [(pleth[0], 125, 30) for pleth in dataset])
    pool.close()
    pool.join()

    new_patient = []
    for i in range(len(dataset)):
        temp = []
        temp.append(result[i])
        temp.append(dataset[i][1])
        new_patient.append(temp)

    return new_patient


def prepare_modeling(dataset=None, batchsize=None):
    print(f'Prepare modeling')
    pleths = []
    resps = []
    for ppg, rr in dataset:
        pleths.append(ppg.astype(np.float32))
        resps.append(rr)
    pleths = np.asarray(pleths)
    resps = np.asarray(resps)
    print(pleths.shape, resps.shape)

    scaler = MinMaxScaler()
    scaled_pleths = np.asarray([scaler.fit_transform(pleth.reshape(-1,1)) for pleth in pleths])
    print(scaled_pleths.shape, type(scaled_pleths[0][0][0]))

    x, y = scaled_pleths[:], resps[:]

    return tf.data.Dataset.from_tensor_slices((x, y)).batch(batchsize)

In [3]:
subjects = pd.read_csv(f'{DATA_PATH}/patients.csv')
patients = subjects.loc[subjects['diagnosis']!='0']
train_patients = patients.sample(frac=1, random_state=42)
# display(patients.head())
# display(train_patients.head())

In [4]:
EPOCHS = 1000
BATCH_SIZE = 256
LR = 0.001
callbacks = [
    EarlyStopping(monitor='val_loss', patience=33),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
]

In [5]:
counter = 1
raw_dataset = {
    'train': [],
    'val': []
}
dataset = {
    'train': [],
    'val': []
}
for train_idx, val_idx in kf.split(train_patients):
    print(f'{counter}th K-fold')
    counter = counter + 1
    # 이렇게 하는 이유는 Train과 Validation을 완벽히 구별시키기 위함이다.
    X_train = preprocessing(train_patients.iloc[train_idx])
    X_val = preprocessing(train_patients.iloc[val_idx])
    print(f'Preprocessing finished: {len(X_train)} / {len(X_val)}')

    raw_dataset['train'].append(X_train)
    raw_dataset['val'].append(X_val)
    
    train_dataset = prepare_modeling(X_train, batchsize=256)
    val_dataset = prepare_modeling(X_val, batchsize=256)

    dataset['train'].append(train_dataset)
    dataset['val'].append(val_dataset)

1th K-fold
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling
Preprocessing finished: 5180 / 1328
Prepare modeling
(5180, 1800) (5180,)
(5180, 1800, 1) <class 'numpy.float32'>


2023-06-07 18:52:55.766281: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14353 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:73:00.0, compute capability: 7.5


Prepare modeling
(1328, 1800) (1328,)
(1328, 1800, 1) <class 'numpy.float32'>
2th K-fold
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling
Preprocessing finished: 5214 / 1294
Prepare modeling
(5214, 1800) (5214,)
(5214, 1800, 1) <class 'numpy.float32'>
Prepare modeling
(1294, 1800) (1294,)
(1294, 1800, 1) <class 'numpy.float32'>
3th K-fold
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessing: Windowing
Init Preprocessing: Resampling
Preprocessing finished: 5182 / 1326
Prepare modeling
(5182, 1800) (5182,)
(5182, 1800, 1) <class 'numpy.float32'>
Prepare modeling
(1326, 1800) (1326,)
(1326, 1800, 1) <class 'numpy.float32'>
4th K-fold
Extract PLETH/RESP
Init Preprocessing: Filtering
Init Preprocessi

In [10]:
import os
import keras
import tensorflow as tf
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Conv1D, Conv1DTranspose, MaxPooling1D, AveragePooling1D, Dense, BatchNormalization, Activation, Add, Flatten, Dropout, Concatenate, LeakyReLU
print(f'Is GPU Avaliable: {tf.config.list_physical_devices("GPU")}')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

DATA_PATH = '/root/Workspace/DataWarehouse/stMary_RRpo'

class DResIncBlock(Model):
    def __init__(self, starting_filter=32, dropout=0, *args, **kwargs):
        super(DResIncBlock, self).__init__(*args, **kwargs)
        self.conv11 = Conv1D(filters=starting_filter, kernel_size=1, strides=1, padding='same')
        self.bn11 = BatchNormalization()

        self.conv21 = Conv1D(filters=starting_filter, kernel_size=1, strides=1, padding='same')
        self.bn21 = BatchNormalization()
        self.conv22 = Conv1D(filters=starting_filter, kernel_size=3, strides=1, dilation_rate=2, padding='same')
        self.bn22 = BatchNormalization()

        self.conv31 = Conv1D(filters=starting_filter, kernel_size=1, strides=1, padding='same')
        self.bn31 = BatchNormalization()
        self.conv32 = Conv1D(filters=starting_filter, kernel_size=3, strides=1, dilation_rate=4, padding='same')
        self.bn32 = BatchNormalization()

        self.conv41 = Conv1D(filters=starting_filter, kernel_size=1, strides=1, padding='same')
        self.bn41 = BatchNormalization()
        self.conv42 = Conv1D(filters=starting_filter, kernel_size=3, strides=1, dilation_rate=8, padding='same')
        self.bn42 = BatchNormalization()


    def call(self, inputs, training=None, mask=None):
        # inputs ?, timestep, features
        x1 = self.conv11(inputs) 
        x1 = self.bn11(x1)
        
        x2 = self.conv21(inputs)
        x2 = self.bn21(x2)
        x2 = Activation('relu')(x2)
        x2 = self.conv22(x2)
        x2 = self.bn22(x2)
        
        x3 = self.conv31(inputs)
        x3 = self.bn31(x3)
        x3 = Activation('relu')(x3)
        x3 = self.conv32(x3)
        x3 = self.bn32(x3)
        
        x4 = self.conv41(inputs)
        x4 = self.bn41(x4)
        x4 = Activation('relu')(x4)
        x4 = self.conv42(x4)
        x4 = self.bn42(x4)
        
        x = Concatenate()([x1, x2, x3, x4]) # ?, input_timestep, features * 4
        outputs = Add()([x, inputs])
        return outputs


class ContractingPath(Model):
    def __init__(self, filters, *args, **kwargs):
        super(ContractingPath, self).__init__(*args, **kwargs)
        self.blocks = [DResIncBlock(starting_filter=filters*(2**i)) for i in range(3)]
        self.dwnconv = [Conv1D(filters=1, kernel_size=2, strides=2, padding='same') for _ in range(3)]
        self.bn = [BatchNormalization() for _ in range(3)]


    def call(self, inputs, training=None, mask=None):
        f_map1 = self.blocks[0](inputs) # ?, 1800, 128(64)
        x1 = self.dwnconv[0](f_map1) 
        x1 = self.bn[0](x1)
        x1 = LeakyReLU(alpha=0.2)(x1) # ?, 900, 128(64)

        f_map2 = self.blocks[1](x1) # ?, 900, 256(128)
        x2 = self.dwnconv[1](f_map2)
        x2 = self.bn[1](x2)
        x2 = LeakyReLU(alpha=0.2)(x2) # ?, 450, 256(128)

        f_map3 = self.blocks[2](x2) # ?, 450, 512(256)
        x3 = self.dwnconv[2](f_map3)
        x3 = self.bn[2](x3)
        x3 = LeakyReLU(alpha=0.2)(x3) # ?, 225, 512(256)

        return x3, f_map1, f_map2, f_map3
    

class ExpandingPath(Model):
    def __init__(self, filters=512, *args, **kwargs):
        super(ExpandingPath, self).__init__(*args, **kwargs)
        self.upconv = [Conv1DTranspose(filters=int(filters/(2**i)), kernel_size=2, strides=2, padding='same') for i in range(3)]
        self.blocks = [DResIncBlock(starting_filter=int((filters/(2**i))/4)) for i in range(3)]
        self.bn = [BatchNormalization() for _ in range(3)]
    

    def call(self, inputs, feature_map, training=None, mask=None):
        # (?, 225, 512)
        x = self.upconv[2](inputs) # (?, 450, 512)
        x = self.bn[2](x)
        x = LeakyReLU(alpha=0.2)(x)
        x = self.blocks[2](x) # (?, 450, 512)
        x = Concatenate()([x, feature_map[2]]) # (?, 450, 1024)

        x = self.upconv[1](x) # (?, 900, 256)
        x = self.bn[1](x)
        x = LeakyReLU(alpha=0.2)(x)
        x = self.blocks[1](x) # (?, 900, 256)
        x = Concatenate()([x, feature_map[1]]) # (?, 900, 512)

        x = self.upconv[0](x) # (?, 1800, 128)
        x = self.bn[0](x)
        x = LeakyReLU(alpha=0.2)(x)
        x = self.blocks[0](x) # (?, 1800, 128)
        x = Concatenate()([x, feature_map[0]]) # (?, 1800, 256)

        return x
    

class RespNet(Model):
    def __init__(self, *args, **kwargs):
        super(RespNet, self).__init__(*args, **kwargs)
        self.cpath = ContractingPath(filters=16)
        self.bottleneck = DResIncBlock(starting_filter=64)
        self.xpath = ExpandingPath(filters=256)
        self.conv1 = Conv1D(filters=1, kernel_size=2, strides=1, padding='valid')
        self.bn1 = BatchNormalization()

        self.d100 = Dense(100, activation='relu')
        self.d50 = Dense(50, activation='relu')
        self.d10 = Dense(10, activation='relu')
        self.d1 = Dense(1)

    @tf.function
    def call(self, inputs, training=None, mask=None):
        x = inputs # ?, 1800, 1
        x, f1, f2, f3 = self.cpath(x) # (?, 225, 512), (?, 1800, 128), (?, 900, 256), (?, 450, 512)
        x = self.bottleneck(x) # (?, 225, 512)
        x = self.xpath(x, (f1, f2, f3)) # (?, 1800, 256)

        x = self.conv1(x) # (?, 1800, 1)
        x = self.bn1(x)
        x = LeakyReLU(alpha=0.2)(x)

        x = self.d100(x)
        x = self.d50(x)
        x = self.d10(x)

        return self.d1(x)

Is GPU Avaliable: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
train_losses = []
val_losses = []
for i in range(5):
    print(f'{i+1}th K-fold')
    # model = ResNet34()
    # model = ResNet()
    # model = Unet()
    model = RespNet()

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
        loss=keras.losses.MeanAbsoluteError(),
        metrics=keras.metrics.MeanAbsoluteError()
    )  

    # callbacks.append(ModelCheckpoint(f'../models/230531-Resnet-L34-stmary-KF{i}', monitor='val_loss', save_best_only=True))
    
    history = model.fit(
        dataset['train'][i],
        epochs=EPOCHS,
        callbacks=callbacks,
        validation_data=dataset['val'][i]
    )

    train_losses.append(min(history.history['loss']))
    val_losses.append(min(history.history['val_loss']))

1th K-fold
Epoch 1/1000


2023-06-07 19:18:00.393134: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/expanding_path_2/d_res_inc_block_18/conv1d_139/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/expanding_path_2/d_res_inc_block_18/conv1d_139/Conv1D/BatchToSpaceND/block_shape' with dtype int32 and shape [1]
	 [[{{node gradients/expanding_path_2/d_res_inc_block_18/conv1d_139/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/expanding_path_2/d_res_inc_block_18/conv1d_139/Conv1D/BatchToSpaceND/block_shape}}]]
2023-06-07 19:18:00.393237: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/expanding_path_2/d_res_inc_block_18/conv1d_139/Conv1D/Bat

21/21 [==============================] - ETA: 0s - loss: 17.4741 - mean_absolute_error: 17.4741

2023-06-07 19:18:54.195015: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1328]
	 [[{{node Placeholder/_1}}]]


21/21 [==============================] - 60s 863ms/step - loss: 17.4741 - mean_absolute_error: 17.4741 - val_loss: 17.6928 - val_mean_absolute_error: 17.6928 - lr: 0.0010
Epoch 2/1000
21/21 [==============================] - 13s 604ms/step - loss: 16.5074 - mean_absolute_error: 16.5074 - val_loss: 16.5602 - val_mean_absolute_error: 16.5602 - lr: 0.0010
Epoch 3/1000
21/21 [==============================] - 13s 606ms/step - loss: 13.9168 - mean_absolute_error: 13.9168 - val_loss: 13.5408 - val_mean_absolute_error: 13.5408 - lr: 0.0010
Epoch 4/1000
21/21 [==============================] - 13s 609ms/step - loss: 8.7755 - mean_absolute_error: 8.7755 - val_loss: 7.0366 - val_mean_absolute_error: 7.0366 - lr: 0.0010
Epoch 5/1000
21/21 [==============================] - 13s 609ms/step - loss: 5.2307 - mean_absolute_error: 5.2307 - val_loss: 4.4480 - val_mean_absolute_error: 4.4480 - lr: 0.0010
Epoch 6/1000
21/21 [==============================] - 13s 609ms/step - loss: 4.6722 - mean_absolute_e

...

21/21 [==============================] - 56s 904ms/step - loss: 17.6166 - mean_absolute_error: 17.6166 - val_loss: 18.2355 - val_mean_absolute_error: 18.2355 - lr: 0.0010
Epoch 2/1000
21/21 [==============================] - 13s 619ms/step - loss: 17.2851 - mean_absolute_error: 17.2851 - val_loss: 17.7185 - val_mean_absolute_error: 17.7185 - lr: 0.0010
Epoch 3/1000
21/21 [==============================] - 13s 620ms/step - loss: 16.1840 - mean_absolute_error: 16.1840 - val_loss: 16.1824 - val_mean_absolute_error: 16.1824 - lr: 0.0010
Epoch 4/1000
21/21 [==============================] - 13s 620ms/step - loss: 13.5496 - mean_absolute_error: 13.5496 - val_loss: 12.0683 - val_mean_absolute_error: 12.0683 - lr: 0.0010
Epoch 5/1000
21/21 [==============================] - 13s 621ms/step - loss: 8.6770 - mean_absolute_error: 8.6770 - val_loss: 6.8594 - val_mean_absolute_error: 6.8594 - lr: 0.0010
Epoch 6/1000
21/21 [==============================] - 13s 619ms/step - loss: 5.4256 - mean_absolu

2023-06-07 20:08:58.286080: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5194]
	 [[{{node Placeholder/_1}}]]
2023-06-07 20:09:01.706234: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/expanding_path_6/d_res_inc_block_46/conv1d_351/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/expanding_path_6/d_res_inc_block_46/conv1d_351/Conv1D/BatchToSpaceND/block_shape' with dtype int32 and shape [1]
	 [[{{node gradients/expanding_path_6/d_res_inc_block_46/conv1d_351/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/expanding_path_6/d_res_inc_block_46/conv1d_351/Conv1D/BatchToSpaceND/blo

21/21 [==============================] - ETA: 0s - loss: 17.7556 - mean_absolute_error: 17.7556

2023-06-07 20:09:49.369906: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1314]
	 [[{{node Placeholder/_1}}]]


21/21 [==============================] - 56s 929ms/step - loss: 17.7556 - mean_absolute_error: 17.7556 - val_loss: 17.3138 - val_mean_absolute_error: 17.3138 - lr: 0.0010
Epoch 2/1000
21/21 [==============================] - 13s 611ms/step - loss: 17.3713 - mean_absolute_error: 17.3713 - val_loss: 16.7718 - val_mean_absolute_error: 16.7718 - lr: 0.0010
Epoch 3/1000
21/21 [==============================] - 13s 614ms/step - loss: 16.3850 - mean_absolute_error: 16.3850 - val_loss: 15.1112 - val_mean_absolute_error: 15.1112 - lr: 0.0010
Epoch 4/1000
21/21 [==============================] - 13s 616ms/step - loss: 13.5707 - mean_absolute_error: 13.5707 - val_loss: 10.3008 - val_mean_absolute_error: 10.3008 - lr: 0.0010
Epoch 5/1000
21/21 [==============================] - 13s 615ms/step - loss: 8.4475 - mean_absolute_error: 8.4475 - val_loss: 6.1382 - val_mean_absolute_error: 6.1382 - lr: 0.0010
Epoch 6/1000
21/21 [==============================] - 13s 616ms/step - loss: 5.2683 - mean_absolu

In [12]:
train_losses = np.asarray(train_losses)
print(f'{np.mean(train_losses)} ± {np.std(train_losses)}')

4.402172565460205 ± 0.16456861606367038


In [13]:
val_losses = np.asarray(val_losses)
print(f'{np.mean(val_losses)} ± {np.std(val_losses)}')

4.496534633636474 ± 0.44861632075728525
